# Week 1 - Measuring Meaning & Sampling
This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze so that we can begin to build our sample. This notebook outlines scraping text from the web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Next, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis. Finally, we ask you to begin building a corpus for preliminary analysis and articulate what your sample represents in context of your final project.

We made a python package just for this course: lucem_illud. If you haven't installed this package, you should run the following code first. You don't need to install the package later; all you need to do is just to import the package with: import lucem_illud. For your final projects, you may find it useful to [read the lucem_illud source code](https://github.com/UChicago-Computational-Content-Analysis/lucem_illud/tree/main/lucem_illud) and modify your code for your own interests.

In [4]:
!pip install git+https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git
#installing lucem_illud package
#lucem_illud is a Latin phrase meaning "that light", the insight we can discover in text data!
#If you get an error like "Access is denied", try running the `pip` command on the command line as an administrator.

  Cloning https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to /tmp/pip-req-build-nqelm3l9
  Running command git clone --filter=blob:none --quiet https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git /tmp/pip-req-build-nqelm3l9
  Resolved https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to commit 9fa6f1abdc19d159d587b4017ee6897fe7a18d4d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━

If you're not familiar with jupyter notebook, you may wonder what the exclamation mark(!) at the beginning of the command does (or even what pip means). The exclamation mark enables us to execute Terminal commands in the notebook cells (e.g., run `!ls` to display files in the current folder).

There is also a special download required by the `lucem_illud` module in the module `spacy`. You will see this 'en' module later, but you should probably run the following 2 lines of code so you can import `lucem_illud`.

In [2]:
import spacy
!python -m spacy download en

2024-01-08 03:43:14.185657: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 03:43:14.186335: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 03:43:14.188641: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 03:43:16.012275: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.8 MB/s eta 0:00:00
✔ Download and installation successf

For this notebook we will be using the following packages:

In [5]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [78]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [74]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [80]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Content analysis - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-cli

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

But before we proceed to Beautiful Soup, a digression about Python syntax, especially about objects and functions.
For those who are not familiar with the syntax of python (or, if you're familiar with R programming), you might wonder what requests.get or wikiContentRequest.text mean. To understand this, you need to first understand what objects are. You may have heard that Python is an object oriented programming language (unlike the procedure oriented programming language, an example of which is R). Object is a set of variables (or, data) and functions into which you pass your data. So, in object oriented programming languages, like python, variables and functions are bunleded into objects.

For example, let's look at wikiContentRequest. We use dir() function, which returns the list of attributes and functions of objects.

In [81]:
 dir(wikiContentRequest)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

There's 'text' here. We used 'wikiContentRequest.text' to access 'text.' In other words, we use .(dot notation) to access functions from objects. wikiContentRequest has a set of functions, as shown above, and we used 'wikiContentRequest.text' to access one of them. By the way, dot notations do not necessarily refer to functions--it refers to anything that the entity contains.



Moving on to the next step: BeautifulSoup, a Python library which extracts data from HTML and XML, and transforms HTML files into Python objects.

In [82]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia




































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom arti


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [83]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

open() is a function which literally opens and returns the file. This function has multiple modes, and, here, we used mode as 'w', which means: open a file for writing. And then, we use 'write' function to write on the empty file (content_analysis_save) that we created using open(content_analysis_save, mode='w', encoding='utf-8').} What did we write on this file? The text we got from wikiContentRequest.text

Now let's open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag.

In [84]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is the study of documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is their non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in content analysis to au

Another excursion for those who are not familiar with programming: for loop. For loop is used to iterate over a sequence. "ContentPTags" contains multiple paragraphs, each of which starts and ends with `<p>`. What the "for pTag in contentPtags[:3]" does here is: find each paragraph in contentPTags, which, here, we limited to the first three using contentPtags[:3], and then print each paragraph. So, we have three paragraphs. By the way, you can insert `<p>` in juputer notebook!

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [85]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is the study of documents and...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   Quantitative content analysis highlights frequ...
7   Siegfried Kracauer provides a critique of quan...
8   The data collection instrument used in content...
9   According to current standards of good scienti...
10  Furthermore, the Database of Variables for Con...
11  With the rise of common computing facilities l...
12  By having contents of communication available ...
13  Computer-assisted analysis can help with large...
14  Robert Weber notes: "To make valid inferences ...
15  According to today's reporting standards, quan...
16  There are five types of texts in content analy...
17  Content analysis is rese

Since we learned how to do for loop, you might get what we did here: using contentParagraphs = [], we made an empty list; and then, for each paragraph in contentPTags, we substituted every [\d+\] with '', i.e., removed every [\d+\], and then appended each paragraph (now without [\d+\]) to the empty list. As we can see, we have a dataframe, each row of which is each paragraph of contentPTags, without reference indicators.

By the way, what does [\d+\] mean? If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

Now we have a `DataFrame` containing all relevant text from the page ready to be processed

In [86]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [87]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [88]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <font color="red">Exercise 1</font>
<font color="red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</font>

The webcontent I'm going download and analysis is the wikipedia page of "Misogyny." I believe that know more about the definition and discussion of this idea will hellp me know better what perspective should I focus on the final project.

1. Download the webpage

In [89]:
#w_misogyny = 'https://en.wikipedia.org/wiki/Misogyny'
w_misogyny = 'https://en.wikipedia.org/wiki/Misogyny'
requests.get(w_misogyny)

<Response [200]>

In [90]:
w_m_request = requests.get(w_misogyny)
print(w_m_request.text[:1000])

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Misogyny - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-

In [12]:
w_m_soup = bs4.BeautifulSoup(w_m_request.text, 'html.parser')
print(w_m_soup.text[:200])





Misogyny - Wikipedia



































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout 


In [53]:
#misogyny_save = 'w_misogyny.html'
misogyny_save = 'w_misogyny.html'

with open(misogyny_save, mode='w', encoding='utf-8') as f:
    f.write(w_m_request.text)

2. Catch the 'p' from the original webpage

In [132]:
w_m_PTags = w_m_soup.body.findAll('p')
for pTag in w_m_PTags[:50]:
    print(pTag.text)



Misogyny (/mɪˈsɒdʒɪni/) is hatred of, contempt for, or prejudice against women or girls. It is a form of sexism that can keep women at a lower social status than men, thus maintaining the social roles of patriarchy. Misogyny has been widely practised for thousands of years. It is reflected in art, literature, human societal structure, historical events, mythology, philosophy, and religion worldwide.

An example of misogyny is violence against women, which includes domestic violence and, in its most extreme forms, misogynist terrorism and femicide. Misogyny also often operates through sexual harassment, coercion, and psychological techniques aimed at controlling women, and by legally or socially excluding women from full citizenship. In some cases, misogyny rewards women for accepting an inferior status.

Misogyny can be understood both as an attitude held by individuals, primarily by men, and as a widespread cultural custom or system. Sometimes misogyny manifests in obvious and bold 

In [133]:
w_m_paragraphs = []
for pTag in w_m_PTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    w_m_paragraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
w_m_DF = pandas.DataFrame({'paragraph-text' : w_m_paragraphs})
print(w_m_DF)

                                       paragraph-text
0                                                  \n
1   Misogyny (/mɪˈsɒdʒɪni/) is hatred of, contempt...
2   An example of misogyny is violence against wom...
3   Misogyny can be understood both as an attitude...
4   In feminist thought, misogyny also includes th...
..                                                ...
86  In September 2018, it was announced that the L...
87  In October 2018, two senior police officers, S...
88  In September 2020 the Law Commission proposed ...
89  A Home Office spokesperson in October 2021 sta...
90  Camille Paglia, a self-described "dissident fe...

[91 rows x 1 columns]


3. Use Five regular expressions to extract the content



> 3.1 find the sentences include "sexism"







In [151]:
#find the relation between misogyny and sexism
#find the sentences include "sexism"
findSexism = r'[A-Z][^.]*sexism[^.]*?\.'

# Iterate over rows in the DataFrame
for index, row in w_m_DF.iterrows():
    # Access the text in the 'paragraph-text' column
    text_to_search = row['paragraph-text']

    # Use regular expression to find the word 'sexism' in the text
    sexism_result = re.search(findSexism, text_to_search)
    # Use another regex to extract the sentence starting with A-Z and ending with "."
    if sexism_result:
        print(sexism_result.group())

It is a form of sexism that can keep women at a lower social status than men, thus maintaining the social roles of patriarchy.
The American Merriam-Webster Dictionary distinguishes misogyny, "a hatred of women", from sexism, which denotes sex-based discrimination, and "behavior, conditions, or attitudes that foster stereotypes of social roles based on sex.
Social psychology research[vague] describes overt misogyny as "blatant hostile sexism" that raises resistance in women, as opposed to "manifestations of benevolent sexism" or chivalry that lead women to behave in a manner perpetuating patriarchal arrangements.
Online misogyny includes both individual attempts to intimidate and denigrate women, denial of gender inequity (neosexism), and also coordinated, collective attempts such as vote brigading and the Gamergate antifeminist harassment campaign.
For example: Spanish online discussions show a stronger presence of dominance; Italian misogyny has a plurality of stereotyping and objecti



> 3.2 find the sentences include  "definition" or "define"







In [152]:
#find the definition of misogyny
#find the sentences include "definition" or "define"
findSexism = r'[A-Z][^.]*\bdefin(?:e|ition)\b[^.]*\.'

# Iterate over rows in the DataFrame
for index, row in w_m_DF.iterrows():
    # Access the text in the 'paragraph-text' column
    text_to_search = row['paragraph-text']

    # Use regular expression to find the word 'sexism' in the text
    definition_result = re.search(findSexism, text_to_search)
    # Use another regex to extract the sentence starting with A-Z and ending with "."
    if definition_result:
        print(definition_result.group())

English and American dictionaries define misogyny as "hatred of women" and as "hatred, dislike, or mistrust of women".
In 2012, primarily in response to a speech in the Australian Parliament, the Macquarie Dictionary (which documents Australian English and New Zealand English) expanded its definition to include not only hatred of women but also "entrenched prejudices against women".
Manne finds the traditional "hatred of women" definition of misogyny too simplistic, noting it does not account for how perpetrators of misogynistic violence may love certain women; for example, their mothers.
The Macquarie Dictionary revised its definition in 2012 to better match the way the word has been used over the prior 30 years.
At the time of the Law Commission's proposals seven police forces in England and Wales classed misogyny as a hate crime, but that definition had not been adopted across the board.




>3.3 Find the heads under "Online Misogyny"



In [153]:
# Find the specific <h2> tag
target_h2_tag = w_m_soup.find('h2', text='Online misogyny')
# Initialize lists to store the text content of <h2> and <h3> tags
h2_texts = []
h3_texts = []
current_h2_text = None  # To keep track of the current <h2> text

# Find all <h2> and <h3> tags
for tag in w_m_soup.find_all(['h2', 'h3']):
    if tag.name == 'h2':
        current_h2_text = tag.text
    elif tag.name == 'h3':
        h3_texts.append((current_h2_text, tag.text))

In [137]:
# Create a DataFrame for <h3> tags
h3_df = pd.DataFrame(h3_texts, columns=['h2-text', 'h3-text'])

print(h3_df)

                                              h2-text  \
0                                    Historical usage   
1                                    Historical usage   
2                                            Religion   
3                                            Religion   
4                                            Religion   
5                                            Religion   
6                                            Religion   
7   Misogynistic ideas among prominent Western thi...   
8   Misogynistic ideas among prominent Western thi...   
9   Misogynistic ideas among prominent Western thi...   
10  Misogynistic ideas among prominent Western thi...   
11  Misogynistic ideas among prominent Western thi...   
12                                           Violence   
13                                    Online misogyny   
14                                    Online misogyny   
15                                    Online misogyny   
16                             

In [154]:
online_misogyny_df = h3_df[h3_df['h2-text'] == 'Online misogyny']
print(online_misogyny_df)

            h2-text               h3-text
13  Online misogyny   Coordinated attacks
14  Online misogyny         Language used
15  Online misogyny                Incels
16  Online misogyny  With white supremacy


3.4 Find the paragraph start with "Online misogyny......"

In [148]:
# Define the pattern to find paragraphs starting with "Online misogyny"
findOnlineMisogyny =  r'(.+?Online misogyny.+?)(?=\n\n|\n?$)'

# Iterate over rows in the DataFrame
for index, row in w_m_DF.iterrows():
    # Access the text in the 'paragraph-text' column
    text_to_search = row['paragraph-text']

    # Use regular expression to find paragraphs containing "Online misogyny"
    online_misogyny_result = re.search(findOnlineMisogyny, text_to_search, flags=re.DOTALL)

    # Print the paragraphs that match the pattern
    if online_misogyny_result:
        print(online_misogyny_result.group())



Misogynistic rhetoric is pervasive online and has grown more aggressive over time. Online misogyny includes both individual attempts to intimidate and denigrate women, denial of gender inequity (neosexism), and also coordinated, collective attempts such as vote brigading and the Gamergate antifeminist harassment campaign. In a paper written for the Journal of International Affairs, Kim Barker and Olga Jurasz discuss how online misogyny can lead to women facing obstacles when trying to engage in the public and political spheres of the Internet due to the abusive nature of these spaces. They also suggest regulations and shut downs of online misogyny through both governmental and non-governmental means.


3.5 Find the connection that link "misogyny" with social media, online platform

In [147]:
# Find the connection that link "misogyny" with social media, online platform
findOnlineMisogyny =  r'(.+?(Twitter|Facebook|Instagram|social media|platform|online community).+?)(?=\n\n|\n?$)'

# Iterate over rows in the DataFrame
for index, row in w_m_DF.iterrows():
    # Access the text in the 'paragraph-text' column
    text_to_search = row['paragraph-text']

    # Use regular expression to find paragraphs containing "Online misogyny"
    o_m_p_result = re.search(findOnlineMisogyny, text_to_search, flags=re.DOTALL)

    # Print the paragraphs that match the pattern
    if o_m_p_result:
        print(o_m_p_result.group())


A 2016 study conducted by the think tank Demos found that the majority of Twitter messages containing the words "whore" or "slut" were advertisements for pornography. Of those that were not, a majority used the terms in a non-aggressive way, such a discussion of slut-shaming. Of those that used the terms "whore" or "slut" in an aggressive, insulting way, about half were women and half were men. Twitter users most frequently targeted by women with aggressive insults were celebrities, such as Beyoncé Knowles.
Incels, or involuntary celibates, is an online community of men who believe they cannot get into heterosexual relationships. They share a common belief that women pick partners based solely on looks, so due to their unattractiveness, they will be alone forever.  Due to this perception of themselves, incels in turn hate women, and believe that men are systematically discriminated against. Incels have a large network of male-oriented websites dedicated to the cyber hate of women, disc


# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [84]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


In [85]:
print(contentPTags)

[<p><b>Content analysis</b> is the study of <a href="/wiki/Document" title="Document">documents</a> and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> One of the key advantages of using content analysis to analyse social phenomena is their non-invasive nature, in contrast to simulating social experiences or collecting survey answers.
</p>, <p>Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of <a href="/wiki/Text_(literary_theory)" title="Text (literary theory)">texts</a> or artifacts which are <a href="/wiki/Coding_(social_sciences)" title="Coding (social sciences)">assigned labels (sometimes called codes)</a> to indicate the presence of interesting, <a href="/wiki/Semant

Another excursion: Why do we use enumerate() here? enumerate() takes a collection, enumerates, and returns an enumate object with both the numbers and the collection. For example, contentPTags (the collection we used here) is comprised of paragraphs. We want the paragraph number of each paragraph. And this is what enumerate() does: it returns the paragraph number and the paragraph.

We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [119]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number,source-paragraph-number,source-paragraph-text
0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,0,None,None
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1,None,None
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2,None,None
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3,None,None
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4,None,None
...,...,...,...,...,...
23761,Recent novels with an adult audience have incl...,https://en.wikipedia.org/wiki/Content_analysis,23761,1,girls
23762,"Since the 1970s, under the influence of femini...",https://en.wikipedia.org/wiki/Content_analysis,23762,1,girls
23763,There have been many comic books and comic str...,https://en.wikipedia.org/wiki/Content_analysis,23763,1,girls
23764,In Japanese animated cartoons and comic books ...,https://en.wikipedia.org/wiki/Content_analysis,23764,1,girls


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['paragraph-number'].plot(kind='hist', bins=20, title='paragraph-number')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='index', y='paragraph-number', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['paragraph-number']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'paragraph-number'}, axis=1)
              .sort_values('paragraph-number', ascending=True))
  xs = counted['paragraph-number']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('paragraph-number', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('paragraph-number')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_5['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_6['paragraph-number'].plot(kind='line', figsize=(8, 4), title='paragraph-number')
plt.gca().spines[['top', 'right']].set_visible(False)

Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [120]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [121]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-text,source,paragraph-number,source-paragraph-number,source-paragraph-text
0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,0,None,None
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1,None,None
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2,None,None
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3,None,None
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4,None,None
...,...,...,...,...,...
23909,Recent novels with an adult audience have incl...,https://en.wikipedia.org/wiki/Girl,58,1,girls
23910,"Since the 1970s, under the influence of femini...",https://en.wikipedia.org/wiki/Girl,59,1,girls
23911,There have been many comic books and comic str...,https://en.wikipedia.org/wiki/Girl,60,1,girls
23912,In Japanese animated cartoons and comic books ...,https://en.wikipedia.org/wiki/Girl,61,1,girls



# <font color="red">Exercise 2</font>
<font color="red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</font>



In [66]:
#w_ss= 'https://en.wikipedia.org/wiki/Slut-shaming'
w_ss= 'https://en.wikipedia.org/wiki/Slut-shaming'
w_ss_PAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(w_m_PTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        w_ss_PAgeURLS.append((
            urllib.parse.urljoin(w_ss, relurl),
            paragraphNum,
            linkText,
        ))
print(w_ss_PAgeURLS[:10])

NameError: name 'w_m_PTags' is not defined

In [67]:
print(w_m_PTags)

NameError: name 'w_m_PTags' is not defined

In [68]:
w_m_DF['source'] = [w_misogyny] * len(w_m_DF['paragraph-text'])
w_m_DF['paragraph-number'] = range(len(w_m_DF['paragraph-text']))

w_m_DF

NameError: name 'w_misogyny' is not defined

In [69]:
w_m_DF['source-paragraph-number'] = [None] * len(w_m_DF['paragraph-text'])
w_m_DF['source-paragraph-text'] = [None] * len(w_m_DF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    w_Dict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
         w_Dict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
         w_Dict['paragraph-number'].append(parNum)
         w_Dict['source'].append(targetURL)
         w_Dict['source-paragraph-number'].append(sourceParNum)
         w_Dict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(w_Dict)

NameError: name 'w_m_DF' is not defined

In [70]:
for urlTuple in w_ss_PAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    w_m_DF = w_m_DF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
w_m_DF

NameError: name 'w_m_DF' is not defined

## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [71]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[],"uuid":"t:nXFqyQsaizVnIxVAm-ttlA"},"posts-start":0,"posts-total":3926,"posts-type":false,"posts":[{"id":"679413944568430592","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/679413944568430592","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/679413944568430592\/cat-cats-kitty-gato-saturday-meow-katze","type":"photo","date-gmt":"2022-03-22 09:00:29 GMT","date":"Tue, 22 Mar 2022 10:00:29","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1647939629,"format":"html","reblog-key":"MDkw6o5C","slug":"cat-cats-kitty-gato-saturday-meow-katze","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"679413944568430592\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_679413944568430592\"><iframe id=\"like_iframe_679413944568430592\" src=\"https:\/\/assets.tumblr.com\/

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [72]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [73]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [74]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,id,photo-url,date,tags,photo-type
0,679413944568430592,https://64.media.tumblr.com/1fb7e346f39e428540...,"Tue, 22 Mar 2022 10:00:29","[gif, lolcat, lolcats, cat, funny, cats, kitty...",gif
1,662815854023655425,https://64.media.tumblr.com/021eac8fbcafbb00a5...,"Mon, 20 Sep 2021 06:00:56","[gif, lolcat, lolcats, cat, funny]",gif
2,662778109891952640,https://64.media.tumblr.com/8c0517adb8c71e4a3d...,"Sun, 19 Sep 2021 20:01:00","[cat, cats, lol, lolcat, lolcats]",png
3,662657302700146688,https://64.media.tumblr.com/061d27cda309d5c809...,"Sat, 18 Sep 2021 12:00:50","[cat, cats, lol, lolcat, lolcats]",jpg
4,662513901538246656,https://64.media.tumblr.com/80584a9d1ff4ddc4fc...,"Thu, 16 Sep 2021 22:01:32","[cat, cats, lol, lolcat, lolcats]",png
5,662257177983090688,https://64.media.tumblr.com/893b320cd2e8970a20...,"Tue, 14 Sep 2021 02:01:01","[cat, cats, lol, lolcat, lolcats]",jpg
6,662166591527698432,https://64.media.tumblr.com/c7f0a0a9184e480e15...,"Mon, 13 Sep 2021 02:01:11","[cat, cats, lol, lolcat, lolcats]",jpg
7,662113740899090432,https://64.media.tumblr.com/07f7be7f71917a6049...,"Sun, 12 Sep 2021 12:01:09","[cat, cats, lol, lolcat, lolcats]",jpg
8,661955166248026112,https://64.media.tumblr.com/205f030c48d31f8960...,"Fri, 10 Sep 2021 18:00:40","[cat, cats, lol, lolcat, lolcats]",jpg
9,661894830378614784,https://64.media.tumblr.com/c463bff883fec2045b...,"Fri, 10 Sep 2021 02:01:39","[cat, cats, lol, lolcat, lolcats]",png


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [75]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [76]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. Lets start by looking at the Shakespeare files in the `data` directory

In [91]:
with open('./data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

FileNotFoundError: [Errno 2] No such file or directory: './data/Shakespeare/midsummer_nights_dream.txt'

By the way, depending on your working directory, you might get errors such as: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt.' Don't panic, it's nothing, just check your working directory.

Then to load all the files in `./data/Shakespeare` we can use a for loop with `scandir`:

In [10]:
targetDir = './data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

FileNotFoundError: [Errno 2] No such file or directory: './data/Shakespeare'

Then we can put them all in pandas DataFrame

In [ ]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [41]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.3
%���������
4 0 obj
<< /Length 5 0 R /Filter /FlateDecode >>
stream
x�]۶�F�}�W�c����T���C�i�<t�bM�f
Tn�<3_��CDf��J�N�i��#�%.;.���	?߄��7�]8������ux��}޾m����y��bǾ����!���$�Ǯ���C��F�����p���5��1��1�P<�{�$��/$�P�s�v��PgH?�����Q�~�*�:l��ˇ�m�ǰ��C�l����܊��E�����!�^�y��m�$�Ý���wۡل׼�6w���ī�K�~؞���r��~?�ˡkO�;6IH�9{ԡ��� ]?�E�E��~���.l������+��W�_�����C��S�|�~C��N�3ӛB`8�ޚj9���AZ��0�d�l^������SY	�Ƨ��>q�ۇ&
����.�����0����; ��>a8�$w�p��p����ST���. �7��@����)��&1�|���WՃ jOv�G2b�L8I��N�@gǍ������O�C��������IN@@�����}�8��+L����a�&ү�o�V(���0���+5�
SfS&��<�2�����>l�V��&��=4⇤=�W��<�JMo����"����d�C����[vY�|K{_ܔ\����%H�/@'�QA�+D�l��c��L�G�.��	�̎�V�:f>���AwK���o$`D��bE45�0�%th6h�����>*�2vQd�+M��Y}�Q���u�[���N�o'b��/u�.r'Z���J�e8�v��;��{T�	�����^8�  l<�E�<���b�����C8j��f��xB>K�����|w��f�|?�s̭��Y�'�Ip&�"�A���f�?�!IYi���U�"��y;���#��e3)�+B�&����<E9I�g�/]"D��yfC;e����Y^�z ��s'�)/�X�-HY��<ˬ�ݰ

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [42]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [43]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [44]:
print(readPDF(infoExtractionBytes)[:550])

Department of  Sociology 

THE UNIVERSITY OF CHICAGO 

SOCIOLOGY 40133 

Computational Content Analysis 

Friday 1:00 – 3:50pm 
Winter 2017-2018 
Classroom: Harper Memorial 130       
http://chalk.uchicago.edu/ 

 

                                                                                           

          Office: McGiffert 210 
                                                    Tel.: 834-3612; jevans@uchicago.edu 
                                  Office Hours: Thursday 12:30-2:30pm 

     

        James A. Evans            

    


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [ ]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud` so we can use it later without having to retype it.

In [ ]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [ ]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [62]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

NameError: name 'downloadIfNeeded' is not defined

Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.

# <font color="red">Exercise 3</font>
<font color="red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</font>


In [46]:
#online_misogyny_pdf = 'https://aclanthology.org/2021.acl-long.247v2.pdf'
online_misogyny_pdf = 'https://aclanthology.org/2021.acl-long.247v2.pdf'


o_m_Request = requests.get(online_misogyny_pdf, stream=True)
pdf_content = io.BytesIO(o_m_Request.content)
print(o_m_Request.text[:1000])

%PDF-1.5
%����
935 0 obj
<< /Linearized 1 /L 596473 /H [ 5009 257 ] /O 940 /E 87776 /N 17 /T 590591 >>
endobj
                                                                                                         
936 0 obj
<< /Type /XRef /Length 80 /Filter /FlateDecode /DecodeParms << /Columns 5 /Predictor 12 >> /W [ 1 3 1 ] /Index [ 935 205 ] /Info 319 0 R /Root 937 0 R /Size 1140 /Prev 590592                /ID [<3c8c41a348fe9761ba3f46026c38de69><1e322b299b68e55466aefb16c86f3515>] >>
stream
 ��H2Flf`b��Wl�(9J�O�,x7��C� ��
endstream
endobj
                                                                                                                                                                                                                                                                                                                                                                                                                  


In [55]:
!pip install pymupdf
import pandas as pd
from io import StringIO
import fitz



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 20.0 MB/s eta 0:00:00


In [56]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str

In [57]:
o_m_Bytes = io.BytesIO(o_m_Request.content)

In [58]:
print(readPDF(o_m_Bytes)[:1000])

Annotating Online Misogyny

IT University of Copenhagen

IT University of Copenhagen

Philine Zeinert

Denmark

Nanna Inie

Denmark

Leon Derczynski

IT University of Copenhagen

Denmark

phze@itu.dk

nans@itu.dk

leod@itu.dk

Abstract

Online misogyny, a category of online abusive
language, has serious and harmful social con-
sequences. Automatic detection of misogynis-
tic language online, while imperative, poses
complicated challenges to both data gathering,
data annotation, and bias mitigation, as this
type of data is linguistically complex and di-
verse. This paper makes three contributions
in this area: Firstly, we describe the detailed
design of our iterative annotation process and
codebook. Secondly, we present a comprehen-
sive taxonomy of labels for annotating misog-
yny in natural written language, and ﬁnally, we
introduce a high-quality dataset of annotated
posts sampled from social media posts.

1

Introduction

Abusive language is a phenomenon with serious
consequences fo

In [61]:
import pandas as pd
from io import StringIO

# Extract text from the PDF
pdf_text = readPDF(o_m_Bytes)

# Create a DataFrame with a single column named 'text'
o_m_df = pd.DataFrame({'text': [pdf_text]})

# Display the DataFrame
print(o_m_df)


                                                text
0  Annotating Online Misogyny\n\nIT University of...


# <font color="red">Exercise 4</font>

<font color="red">In the two cells immediately following, describe a possible project (e.g., it might end up being your final project, but need not be if you are still searching): **WHAT** you will analyze--the texts you will select and the social game, world and actors you intend to learn about through your analysis (<100 words); **WHY** you will analyze these texts to learn about that context--justify the rationale behind your proposed sample design for this project, based on the readings. What is the social game, social work, or social actors about whom you are seeking to make inferences? What are the virtues of your proposed sample with respect to your research questions? What are its limitations? What are alternatives? What would be a reasonable path to "scale up" your sample for further analysis (i.e., high-profile publication)? (<150 words)? [**Note**: your individual or collective project will change over the course of the quarter as new data and/or analysis opportunities arise or if old ones fade away.]

## ***What?***
<100 words

I plan to analyze the phenomenon of misogyny on various social platforms, and I plan to focus on the use of misogynistic words and contextual language situations. For example, what are the words that are commonly used against women on various platforms, and what are the emotional words used by the author when articles related to women appear. Social platforms expected to participate in the analysis include: twitter, instagram, facebook.


## ***Why?***
<150 words

You can know how people view a particular issue based on word usage frequency, methods, context, and emotions. Therefore, through this analysis, we can understand under what circumstances there will be differences on the same topic and whether different platforms bring different word usage strategies to the publishers.
Difficulties that can be foreseen include: Compared with Chinese social networks, the author needs to become more familiar with the common misogynistic words in the Internet ecosystem of English users. Difficulties in fully retrieving relevant articles (anticipate searching with specific keywords, but avoiding omissions as much as possible will be challenging). In addition to using keywords to search for articles, it is also necessary to analyze the context of the misogynistic words and the author's context. What language model strategy to use still needs to be determined.

### Sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg

